<a href="https://colab.research.google.com/github/2003UJAN/SIH_RE-DACT/blob/main/Re_DACT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import spacy
from transformers import pipeline
import torch
import cv2
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
input_text = """
Ujan Pradhan, a student at SRM University, Chennai, India, was born on October 1, 2003.
His email is pradhanujan2003@gmail.com, and he loves to eat and play football.
He works at Samsung R&D Institute in Bengaluru, India as an Intern.
"""

In [9]:
def perform_ner(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities

entities = perform_ner(input_text)
print("Detected Entities:", entities)

Detected Entities: [('Ujan Pradhan', 'PERSON'), ('SRM University', 'ORG'), ('Chennai', 'ORG'), ('India', 'GPE'), ('October 1, 2003', 'DATE'), ('Samsung R&D Institute', 'ORG'), ('Bengaluru', 'GPE'), ('India', 'GPE'), ('Intern', 'FAC')]


In [10]:
def redact_text(text, entities, degree):
    for entity, label in entities:
        if degree == 1:
            if label in ["PERSON", "ORG"]:
                text = text.replace(entity, "[REDACTED]")
        elif degree == 2:
            if label in ["PERSON", "ORG", "DATE"]:
                text = text.replace(entity, "[REDACTED]")
        elif degree == 3:
            text = text.replace(entity, "[REDACTED]")
    return text

In [12]:
redacted_text = redact_text(input_text, entities, degree=2)
print("Redacted Text:", redacted_text)

Redacted Text: 
[REDACTED], a student at [REDACTED], [REDACTED], India, was born on [REDACTED].
His email is pradhanujan2003@gmail.com, and he loves to eat and play football.
He works at [REDACTED] in Bengaluru, India as an Intern.



In [19]:
generator = pipeline("text-generation", model="gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [20]:
def generate_synthetic_data(prompt, max_length=50):
    synthetic_data = generator(prompt, max_length=max_length, num_return_sequences=1)
    return synthetic_data[0]['generated_text']

In [21]:
prompt = "Employee of Samsung R&D Institute, born on October 1, 2003, with email"
synthetic_data = generate_synthetic_data(prompt)
print("Synthetic Data:", synthetic_data)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Synthetic Data: Employee of Samsung R&D Institute, born on October 1, 2003, with email address "gqmck@gmail.com". A native of Germany, the company has more than 14,000 employees worldwide and is one of the top


In [24]:
true_labels = ["PERSON", "ORG", "DATE"]
pred_labels = ["PERSON", "ORG", "DATE"]

precision = precision_score(true_labels, pred_labels, average='macro')
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')

print(f"Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

Precision: 1.0, Recall: 1.0, F1-Score: 1.0


In [25]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 135.4 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 13.0s, installed 2 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-8-23 Python-3.10.12 torch-2.3.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 73.8MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [30]:
def redact_image_with_yolo(image_path, output_path, degree=1):

    image = cv2.imread("/content/Ujan_photo_1.jpeg")

    results = model(image)

    for result in results.xyxy[0]:
        x1, y1, x2, y2, conf, cls = map(int, result)

        if degree == 1:
            face = image[y1:y2, x1:x2]
            face = cv2.GaussianBlur(face, (99, 99), 30)
            image[y1:y2, x1:x2] = face
        elif degree == 2:
            face = image[y1:y2, x1:x2]
            face = cv2.resize(face, (face.shape[1] // 10, face.shape[0] // 10), interpolation=cv2.INTER_LINEAR)
            face = cv2.resize(face, (x2 - x1, y2 - y1), interpolation=cv2.INTER_NEAREST)
            image[y1:y2, x1:x2] = face
        elif degree == 3:
            image[y1:y2, x1:x2] = 0

    cv2.imwrite(output_path, image)

redact_image_with_yolo("Ujan_photo_1.jpeg", "redacted_output.jpg", degree=2)

In [27]:
def redact_video_with_yolo(video_path, output_path, degree=1):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)

        for result in results.xyxy[0]:
            x1, y1, x2, y2, conf, cls = map(int, result)

            if degree == 1:
                face = frame[y1:y2, x1:x2]
                face = cv2.GaussianBlur(face, (99, 99), 30)
                frame[y1:y2, x1:x2] = face
            elif degree == 2:
                face = frame[y1:y2, x1:x2]
                face = cv2.resize(face, (face.shape[1] // 10, face.shape[0] // 10), interpolation=cv2.INTER_LINEAR)
                face = cv2.resize(face, (x2 - x1, y2 - y1), interpolation=cv2.INTER_NEAREST)
                frame[y1:y2, x1:x2] = face
            elif degree == 3:
                frame[y1:y2, x1:x2] = 0

        out.write(frame)

    cap.release()
    out.release()

redact_video_with_yolo("input.mp4", "redacted_output.mp4", degree=2)